# Retrain a CNN, part 2.2, using bottleneck features
* https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
import matplotlib.pylab as plt
import numpy as np

In [4]:
from distutils.version import StrictVersion

In [5]:
import sklearn
print(sklearn.__version__)

assert StrictVersion(sklearn.__version__ ) >= StrictVersion('0.18.1')

0.18.1


In [6]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)
print(tf.__version__)

assert StrictVersion(tf.__version__) >= StrictVersion('1.1.0')

1.2.1


In [7]:
import keras
print(keras.__version__)

assert StrictVersion(keras.__version__) >= StrictVersion('2.0.0')

Using TensorFlow backend.


2.0.6


This script goes along the blog post
"Building powerful image classification models using very little data"
from blog.keras.io.
It uses data that can be downloaded at:
https://www.kaggle.com/c/dogs-vs-cats/data
In our setup, we:
- created a data/ folder
- created train/ and validation/ subfolders inside data/
- created cats/ and dogs/ subfolders inside train/ and validation/
- put the cat pictures index 0-999 in data/train/cats
- put the cat pictures index 1000-1400 in data/validation/cats
- put the dogs pictures index 12500-13499 in data/train/dogs
- put the dog pictures index 13500-13900 in data/validation/dogs
So that we have 1000 training examples for each class, and 400 validation examples for each class.
In summary, this is our directory structure:
```
data/
    train/
        dogs/
            dog001.jpg
            dog002.jpg
            ...
        cats/
            cat001.jpg
            cat002.jpg
            ...
    validation/
        dogs/
            dog001.jpg
            dog002.jpg
            ...
        cats/
            cat001.jpg
            cat002.jpg
            ...
```

In [8]:
!ls -lh data

total 8.0K
drwxrwxr-x 4 ubuntu ubuntu 4.0K Aug 31 18:52 train
drwxrwxr-x 4 ubuntu ubuntu 4.0K Aug 31 18:52 validation


In [11]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications

# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 50
batch_size = 16

In [12]:
# build the VGG16 network
model = applications.VGG16(include_top=False, weights='imagenet')

### Next step is to use those saved bottleneck feature activations and train our own, very simple fc layer

In [15]:
train_data = np.load(open('bottleneck_features_train.npy', 'rb'))

In [24]:
train_data.shape[1:]

(4, 4, 512)

In [17]:
# first half of data is dog (0), second half is cat (1)
train_labels = np.array(
    [0] * (nb_train_samples // 2) + [1] * (nb_train_samples // 2))

In [20]:
# same for validation
validation_data = np.load(open('bottleneck_features_validation.npy', 'rb'))
validation_labels = np.array(
    [0] * (nb_validation_samples // 2) + [1] * (nb_validation_samples // 2))

In [21]:
model = Sequential()
model.add(Flatten(input_shape=train_data.shape[1:]))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy', metrics=['accuracy'])

In [22]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               2097408   
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 257       
Total params: 2,097,665
Trainable params: 2,097,665
Non-trainable params: 0
_________________________________________________________________


In [25]:
model.fit(train_data, train_labels,
          epochs=epochs,
          batch_size=batch_size,
          validation_data=(validation_data, validation_labels))

Train on 2000 samples, validate on 800 samples
Epoch 1/50
2000/2000 [==============================] - 1s - loss: 0.8996 - acc: 0.7615 - val_loss: 0.3003 - val_acc: 0.8675
Epoch 2/50
2000/2000 [==============================] - 0s - loss: 0.3514 - acc: 0.8585 - val_loss: 0.3761 - val_acc: 0.8375
Epoch 3/50
2000/2000 [==============================] - 0s - loss: 0.2859 - acc: 0.8860 - val_loss: 0.2506 - val_acc: 0.8988
Epoch 4/50
2000/2000 [==============================] - 0s - loss: 0.2573 - acc: 0.9115 - val_loss: 0.3732 - val_acc: 0.8575
Epoch 5/50
2000/2000 [==============================] - 0s - loss: 0.2186 - acc: 0.9150 - val_loss: 0.2909 - val_acc: 0.9087
Epoch 6/50
2000/2000 [==============================] - 0s - loss: 0.2290 - acc: 0.9190 - val_loss: 0.3718 - val_acc: 0.8812
Epoch 7/50
2000/2000 [==============================] - 0s - loss: 0.1811 - acc: 0.9305 - val_loss: 0.3277 - val_acc: 0.9038
Epoch 8/50
2000/2000 [==============================] - 0s - loss: 0.1483 - ac

In [26]:
top_model_weights_path = 'bottleneck_fc_model.h5'
model.save_weights(top_model_weights_path)